# Imports

In [1]:
import pandas as pd
import numpy as np
import requests
from PyPDF2 import PdfFileReader, PdfFileMerger, PdfFileWriter
import PyPDF2
from bs4 import BeautifulSoup
import os
import shutil
from urllib.parse import urljoin
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
import cloudinary.uploader
import json
from datetime import date
import sys

In [13]:
reader = PdfFileReader('p_ftj_contas_deposito.pdf')
page = reader.getPage(1)

In [14]:
page.getContents()

[IndirectObject(16, 0),
 IndirectObject(17, 0),
 IndirectObject(18, 0),
 IndirectObject(19, 0),
 IndirectObject(20, 0),
 IndirectObject(21, 0),
 IndirectObject(22, 0),
 IndirectObject(23, 0),
 IndirectObject(24, 0),
 IndirectObject(25, 0),
 IndirectObject(26, 0),
 IndirectObject(27, 0),
 IndirectObject(107, 0),
 IndirectObject(108, 0)]

In [16]:
def file_decrypt(pdf_url, filename="tempe.pdf"):
    """Accepts pdf urls. Returns PyPDF2 FileReader Objects"""
    print(f'file_decrypt was called for {pdf_url}')
    response = requests.get(pdf_url)
    if response.status_code == 200:
        temp = open(filename, "wb")
        temp.write(response.content)
        ## important: have to close again
        temp.close()
        command="cp "+filename+" temp.pdf; qpdf --password='' --decrypt temp.pdf "+filename
        os.system(command)
        print('File decrypted (qpdf)')
        #re-open the decrypted file
        pdfFile = PdfFileReader(filename)
        # removing temp file
        os.remove(filename)
    else: 
        print(f'{url} could not be reached with file_decrypt. Response: {response}')
        return None
    return pdfFile

In [17]:
file = file_decrypt('https://banco.bankinter.pt/particulares/pdfs/precario/p_ftj_contas_deposito.pdf')
page = file.getPage(1)
page.extractText()

file_decrypt was called for https://banco.bankinter.pt/particulares/pdfs/precario/p_ftj_contas_deposito.pdf
File decrypted (qpdf)


''

In [20]:
page.getContents()

[IndirectObject(19, 0),
 IndirectObject(20, 0),
 IndirectObject(21, 0),
 IndirectObject(22, 0),
 IndirectObject(23, 0),
 IndirectObject(24, 0),
 IndirectObject(25, 0),
 IndirectObject(26, 0),
 IndirectObject(27, 0),
 IndirectObject(28, 0),
 IndirectObject(29, 0),
 IndirectObject(30, 0),
 IndirectObject(31, 0),
 IndirectObject(32, 0)]

# Testing own api

In [32]:
url = 'http://bank-price-api.herokuapp.com/'
post = {"1":{"url":"https://www.abanca.pt", 
                    "name":"ABANCA Corporación Bancaria, S.A.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0170"}
             ,
              "2":{"url":"https://www.bankinter.pt", 
                    "name":"Bankinter, S.A.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0269"}
              ,
              "3":{"url":"https://www.bancobic.ao", 
                    "name":"Banco BIC, SA", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0079"}
              }


In [30]:
post

{'1': {'url': 'https://www.abanca.pt',
  'name': 'ABANCA Corporación Bancaria, S.A.',
  'num_pdfs': '3',
  'last_updated': '210401',
  'sum_sizes': '40',
  'bp_bank_id': '0170'},
 '2': {'url': 'https://www.bankinter.pt',
  'name': 'Bankinter, S.A.',
  'num_pdfs': '3',
  'last_updated': '210401',
  'sum_sizes': '40',
  'bp_bank_id': '0269'}}

In [31]:
r = requests.post(url, data=post)
r

<Response [405]>

In [63]:
## expected result
result = {
    "1": {
        "bp_bank_id": "0170",
        "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf",
        "last_updated": "210401",
        "list_pdfs": [
            "https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf",
            "https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf",
            "https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf",
            "https://www.abanca.pt/files/documents/glossario-6479e89f.pdf",
            "https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf"
        ],
        "name": "ABANCA Corporación Bancaria, S.A.",
        "num_pdfs": 5,
        "price_page": "https://www.abanca.pt/pt/precario/",
        "sum_sizes": "40",
        "url": "https://www.abanca.pt"
    },
    "2": {
        "bp_bank_id": "0269",
        "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf",
        "last_updated": "210401",
        "list_pdfs": [
            "https://www.bankinter.pt/documents/20182/115991/Pre%C3%A7%C3%A1rio+Completo+TFSI++20210401.pdf/c489f0cd-3831-4441-812e-f351741d2b22",
            "https://banco.bankinter.pt/particulares/pdfs/precario/folha_rosto.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/informacao_geral.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/informacao_complementar.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_contas_deposito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_operacoes_credito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cartoes_credito_debito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cheques.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_transferencias.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cobrancas.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_prestacao_servicos.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_contas_deposito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_operacoes_credito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cartoes_credito_debito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cheques.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_transferencias.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cobrancas.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_prestacao_servicos.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_operacoes_estrangeiro.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_ftj_contas_deposito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/p_ftj_operacoes_credito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_ftj_contas_deposito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/o_ftj_operacoes_credito.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_a.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_b.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_c.pdf",
            "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf"
        ],
        "name": "Bankinter, S.A.",
        "num_pdfs": 27,
        "price_page": "https://www.bankinter.pt/precario1",
        "sum_sizes": "40",
        "url": "https://www.bankinter.pt"
    },
    "3": {
        "bp_bank_id": "0079",
        "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE.pdf",
        "last_updated": "210401",
        "list_pdfs": [],
        "name": "Banco BIC, SA",
        "num_pdfs": 0,
        "price_page": {
            "error": "page not reachable"
        },
        "sum_sizes": "40",
        "url": "https://www.bancobic.ao"
    }
}

# last_updated return

In [34]:
pdf = "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_contas_deposito.pdf"
remoteFile = urlopen(Request(pdf)).read()

In [35]:
memoryFile = BytesIO(remoteFile)
pdfFile = PdfFileReader(memoryFile)

In [39]:
pdfFile.getDocumentInfo()

{'/Author': 'BK10813',
 '/CreationDate': "D:20210331170537+01'00'",
 '/ModDate': "D:20210331170537+01'00'",
 '/Producer': 'Microsoft: Print To PDF',
 '/Title': '01. Contas de DepÃ³sito - FCD - Particulares - 20210401.pdf'}

In [61]:
sys.getsizeof(remoteFile)

721022

In [67]:
for bank_id, vals in result.items():
    pdfs = vals.get('list_pdfs')
    if pdfs == []:
        break
    else:
        filesizes = []
        for pdf in pdfs:
            try:
                remote = urlopen(Request(pdf)).read()
                memory = BytesIO(remote)
                file = PdfFileReader(memory)
                filesizes.append(sys.getsizeof(memory))
            except:
                print(f'cannot reach: {pdf}')

        vals['sum_sizes'] = sum(filesizes)


cannot reach: https://www.abanca.pt/files/documents/glossario-6479e89f.pdf


In [68]:
result

{'1': {'bp_bank_id': '0170',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
  'last_updated': '210401',
  'list_pdfs': ['https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf',
   'https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf',
   'https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf',
   'https://www.abanca.pt/files/documents/glossario-6479e89f.pdf',
   'https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf'],
  'name': 'ABANCA Corporación Bancaria, S.A.',
  'num_pdfs': 5,
  'price_page': 'https://www.abanca.pt/pt/precario/',
  'sum_sizes': 320,
  'url': 'https://www.abanca.pt'},
 '2': {'bp_bank_id': '0269',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf',
  'last_updated': '210401',
  'list_pdfs': ['https://www.bankinter.pt/documents/20182/115991/Pre%C3%A7%C3%A1rio+Completo+TF

# Resolving connection errors

In [80]:
banks = {'bic':{'url':'https://www.bancobic.ao'}}


def find_price_pages(banks=banks, search=['preçário', 'pricelist', 'precario']):    
    search = [x.lower() for x in search]
    for k, v in banks.items():
        url = v.get('url')
        print(f'parsing url: {url}')
        # only look for the pt page
        headers = {'Accept-Language': 'pt-PT'}
        r = requests.get(url, headers=headers)
        print(f'banks response: {r}')
        soup = BeautifulSoup(r.content, 'html.parser')
        print(soup)
#         if r.status_code == 200:
            # going through every link on the page to see if 'precarios' is in the link
        for link in soup.find_all('a', href=True):
            url_prices = str(link.get('href').lower().strip())
            lower = str(link.string).lower().strip()
            title = str(link.get('title')).strip().lower()
            searchstring = ' '.join([url_prices, lower, title])
            if any([x in searchstring for x in search]):
                print(f'found terms of {search} in string {searchstring}')
                # some links in the source code are relative, some are absolute -- using urljoin
                v['pricelist_url'] = urljoin(url,url_prices)
                print(f'added link to banks: {urljoin(url,url_prices)}')
#         else:
#             print(f'could not reach page: {url}')
    return banks

In [81]:
find_price_pages()

parsing url: https://www.bancobic.ao
banks response: <Response [502]>
<!DOCTYPE html>

<html>
<head>
<title>BIC - Em manutenção...</title>
<style>
			html,
			body {
				line-height: normal;
				color: #363636;
				font-size: 12px;
				font-family: Arial, verdana, 'sans-serif';
			}
			
			.maintenance-container {
				text-align: center;
				margin-top: 100px;
			}
			
			.maintenance-image {
				margin-bottom: 40px;
			}
			
			.company-logo {
				margin-top: 40px;
			}
			
			h1, p {
				padding: 0;
				margin: 0;
				line-height: 20px;
				text-align: center;
			}
			
			h1 {
				margin-bottom: 20px;
			}
		</style>
</head>
<body>
<div class="maintenance-container">
<img class="maintenance-image" src="maintenance-icon.png"/>
<h1>Estamos em manutenção</h1>
<p>Pedimos desculpa pelo incomodo, estaremos operacionais o mais rápidamente possível. </p>
<p>Obrigado pela sua compreensão</p>
<img class="company-logo" src="logotype.png"/>
</div>
</body>
</html>


{'bic': {'url': 'https://www.bancobic.ao'}}

# encoding and concatenating urls

In [33]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlencode, quote_plus

In [43]:
url = 'https://www.bancobai.ao'

price_page = 'https://www.bancobai.ao/pt/preçário'
price_page_enc = quote_plus(price_page)
r = requests.get(price_page)
price_page_enc

'https%3A%2F%2Fwww.bancobai.ao%2Fpt%2Fpre%C3%A7%C3%A1rio'

In [41]:
soup = BeautifulSoup(r.content, 'html.parser')
soup


<!DOCTYPE html>

<html lang="pt">
<head prefix="og: http://ogp.me/ns#">
<!--Analytics-->
<!-- Google Tag Manager -->
<script>
            (function (w, d, s, l, i) {
                w[l] = w[l] || []; w[l].push({
                    'gtm.start':
                        new Date().getTime(), event: 'gtm.js'
                }); var f = d.getElementsByTagName(s)[0],
                    j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : ''; j.async = true; j.src =
                        'https://www.googletagmanager.com/gtm.js?id=' + i + dl; f.parentNode.insertBefore(j, f);
            })(window, document, 'script', 'dataLayer', 'GTM-WFRHC2X');</script>
<!-- End Google Tag Manager -->
<!-- Analytics(END) -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>BAI - Preçário</title>
<meta content="" name="description">
<meta

In [42]:
print(f'looking for pdfs in: {price_page}')
pdf = []
for link in soup.find_all('a', href=True):
    
    href = link.get('href')
    if '.pdf' in href:
        # some pdf links are absolute links
        print(href)
        # href = quote_plus(href)
        pdf.append(urljoin(url,href))
        
pdf

looking for pdfs in: https://www.bancobai.ao/pt/preçário
/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
/media/2989/bai_pre_outros-clientes_31-03-2021.pdf
/media/2823/bai_resumo-das-alteracoes-do-precario-bna-8-02-2021-cleaned.pdf
/media/2278/termos-e-condições.pdf


['https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf',
 'https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2021.pdf',
 'https://www.bancobai.ao/media/2823/bai_resumo-das-alteracoes-do-precario-bna-8-02-2021-cleaned.pdf',
 'https://www.bancobai.ao/media/2278/termos-e-condições.pdf']

In [ ]:
            //   ,
            //   "4":{"url":"https://www.bancobai.ao", 
            //         "name":"Banco BAI, SA", 
            //         "num_pdfs":"3", 
            //         "last_updated":"210401", 
            //         "sum_sizes":"40", 
            //         "bp_bank_id":"0008"}